[![Julia](../img/julia-logo-color.png)](https://julialang.org/)

# Seminario. Introducción al lenguaje Julia

## 12. Ámbito de variables

## Objetivos

- Ámbito de una variable
- Ámbito global
- Ámbito local
- Constantes

## Ámbito de una variable

El ámbito de una variable es la región del código en la que ésta es visible. El ámbito permite resolver conflictos entre nombres de variables pudiendo utilizar el mismo identificador para objetos diferentes, por ejemplo con dos funciones que usan el mismo nombre para alguno de sus argumentos. En Julia las regiones de código que marcan el ámbito de una variable están asociadas a las siguientes construcciones

|  **Construcción**  |  **Tipo de ámbito**  |  **Permitido en**  |
| :----------------- | :------------------- | :----------------- | 
| ```module```       |  _global_            |      _global_      |
| ```struct```       |  _local (soft)_      |      _global_      |
| ```for```, ```while```, ```try``` |  _local (soft)_     |  _global_, _local_  |
| ```macro```        |  _local (hard)_     |      _global_      |
| funciones, bloques ```do, let``` |  _local (hard)_     |  _global_, _local_  |

## Ámbito global

Julia permite la programación estructurada pudiendo encapsular código en módulos. Un módulo encapsula un conjunto de variables y funciones. El ámbito global está asociado a los modulos, es decir, tiene su propio ámbito global. Por ejemplo, 

In [ ]:
module A
    a = 1  # Variable global en el ámbito del modulo A
end;

La variable **a** definida en el módulo A, es distinta a cualquier otra variable con el mismo nombre definida en otro ámbito, 

In [ ]:
a = 2
module A
    a = 1  # Variable global en el ámbito del modulo A
end;
println(a)

De esta manera, las funciones definidas en un módulo acceden a las variables globales definidas en él

In [ ]:
module A
    a = 1
    imprime() = a    # se refiere a la variable a del modulo A
end;
a = 2         # Esta variable es distinta a la del modulo A
A.imprime()   # Muestra A.a

Los modulos pueden anidarse, y se puede acceder a las variables globales del módulo "interno" con el acceso
```julia
    *nombre_modulo*.*variable*
```

In [ ]:
module B
    module C
        c = 2 
    end
    b = C.c   # Acceso al contenido del modulo C
end;    

Como se ha mencionado, cada módulo define un nuevo ámbito global, aunque puede usar las variables definidas en otros módulos generalmente mediante las sentencias ```using``` o ```import```. Si en el módulo B queremos acceder a las variables de A, tenemos que importarlo de la siguiente manera

In [ ]:
module B
    module C
        c = 2
    end
    b = C.c      
    import ..A   # Modulo A disponible
    d = A.a
end; 

En los entornos interactivos como el REPL o los IJulia notebooks, todas las variables y módulos se declaran o definen en el módulo ```Main```, por lo que es posible acceder a su contenido directamente usando el acceso cualificado apropiado sin necesidad de utilizar ```import``` o ```using```,

In [ ]:
println("Variable d = $(B.d)") 

In [ ]:
println("Variable c = $(B.C.c)") 

Obsérvese esta otra forma de acceder a la variable **c**,

In [ ]:
import .B.C
C.c

## Ámbito local

Julia permite definir variables locales dentro de bloques de código tipo ```struct```, bucles ```for```, ```while```,..., funciones, etc. Como en otros lenguajes, no es necesario declarar las variables pues la asignación a una nueva variable se asume como una declaración implícita. Aún así, Julia también permite la declaración explícita de variables locales. La sentencia ```local x``` declara una nueva variable local ```x``` independientemente de que exista otra variable en el ámbito global con el mismo nombre.

Ante la sentencia ``` x = <value>``` en un ámbito local, basándose en la posición donde se realiza la asignación y la referencia actual de ```x``` existente, se aplican las siguientes reglas:
- Existe ```x``` local: simplemente se realiza la nueva asignación a ```x```.
- Ámbito duro (hard scope): si la variable local ```x``` no ha sido ya declarada y la sentencia se realiza dentro de funciones, macros u otras construcciones de ámbito duro (ver tabla arriba), entonces se declara la variable local y se asigna. 
- Ámbito blando (Soft scope): si la variable local```x``` no ha sido ya declarada, y todas las construcciones que contienen la asignación son _soft_ (bucles, ```try/catch```, ```struct```, el comportamiento depende de si la variable global ```x``` existe o no,
 - Si ```x``` global no está definida se crea la variable local ```x``` en el ámbito en el que se realiza la asignación.
 - Si ```x``` global está definida, la asignación se considera ambigua:
      - En entornos no interactivos (ficheros, eval), hay un aviso de abigüedad y se crea la variable local.
      - En entornos interactivos (REPL, notebooks), se asigna a la variable global ```x```.

Veamos un primer ejemplo relativo al **hard scope**

In [ ]:
x = 55   # Variable en ámbito global
function rocks()
    x = "Julia rocks"  # Nueva x local
    println(x)
end

La llamada a la función ```rocks()``` muestra el valor de la variable local definida dentro de ella independientemente de que exista una variable global con el mismo nombre (hard scope).

In [ ]:
rocks()   

La variable global no ha sido reasignada y conserva su valor,

In [ ]:
x  # Variable global

Cuando ya existe una variable local, simplemente se reasigna el con el nuevo valor. La siguiente función suma los primeros _n_ números naturales,

In [ ]:
function sum_to(n)
    s = 0        # Nueva variable local
    for i in 1:n
        s += i   # Se reasigna la variable local existente
    end
    return s
end

In [ ]:
sum_to(5)

In [ ]:
s  # No existe fuera de la función

En el ejemplo anterior, aunque el bucle ```for``` tiene su propio ámbito local, como la variable **s** ya existe no se crea una nueva (soft scope) y simplemente se reasigna la existente. Para aclararlo mejor veamos esta variación de la función anterior,

In [ ]:
function sum_to_def(n)
    s = 0        # Nueva variable local
    for i in 1:n
        t = s + i
        s = t   # Se reasigna la variable local existente
    end
    return s, @isdefined(t)
end

In [ ]:
sum_to_def(5)

La variable **t** es local al bucle ```for``` y por tanto no está definida fuera de él.

Las cosas pueden ser un poco más complicadas cuando trabajamos en el ámbito local (soft), es decir, principalmente con variables en bucles. El comportamiento es diferente a si estamos utilizando un entorno interactivo como REPL o notebook(), o bien si estamos directamente ejecutando un fichero (entorno no interactivo). Vamos a extraer el cuerpo de instrucciones de la función ```sum_to``` y comprobaremos qué ocurre cuando ejecutamos interactivamente y cuando lo hacemos ejecutando un fichero,

In [ ]:
s = 0
for i = 1:10 # Otra forma del bucle
    s = s + i
end
s

Trabajando interactivamente, como el bucle ```for``` tiene un ámbito local soft y la variable global **s** existe, directamente se asigna a ésta. Si no existiera, se crearía una nueva variable local en el ámbito del bucle. Por el contrario, veamos en el caso de ejecutar un fichero. Lo haremos definiendo el código en una variable y utilizando la función ```include_string()``` para evitar editar un fichero aparte

In [ ]:
code = """
s = 0
for i = 1:10
    s = s + i
end
""";

In [ ]:
include_string(Main, code)

En este caso ocurren varias cosas:
- Se declara y asigna una variable global **s**
- La asignación _s = s + i_ se realiza en ámbito local **soft**, en un bucle fuera de una función.
- En el bucle ```for``` se hace referencia a la variable **s** local. Como no está definida se crea local al bucle y se da un aviso de ambigüedad.
- Como la variable **s** no estaba definida, hay un error al evaluar la expresión _s = s + i_.

Para subsanar el problema, y teniendo en cuenta la lógica del código que es incrementar la variable global **s**, podemos declarar la variable ```global``` como sugiere el _Warning_,

In [ ]:
code = """
s = 0
for i = 1:10
    global s = s + i
end
""";

In [ ]:
include_string(Main, code)
s

Hay que resaltar nuevamente que esta ambigüedad no se produce en el ámbito local de funciones (hard) ya que en él se ignora cualquier variable global que pudiera existir como se ha visto en ejemplos anteriores. Tampoco en entornos interactivos como el REPL o los Jupyter notebook pues se asume el mismo comportamiento que en funciones, ámbito local _hard_.

Para finalizar, señalar que las variables de iteración en un bucle son siempre locales, como muestra el siguiente ejemplo

In [ ]:
function f()
    i = 0
    for i = 1:3
        # nada
    end
    return i
end
i    

## Constantes

Se declaran con la palabra reservada ```const```. Se utilizan para variables globales en el ámbito global cuyo valor no va a cambiar (aunque en algunos casos se pude hacer...). La declaración de variables globales constantantes ayuda a mejorar la eficiencia del código. Algunos ejemplos,

In [ ]:
const e  = 2.71828182845904523536;

In [ ]:
e

In [ ]:
Si intentamos modificar el valor de una constante nos podemos encontrar con diferentes situaciones,

In [ ]:
e = 2

La asignación anterior no está permitida porque se le asigna un tipo diferente a la constante **e**. En cambio, la siguiente instrucción sí que está permitida aunque no es aconsejable como indica el aviso correspondiente,

In [ ]:
e = 1.0